In [1]:
import pandas as pd
import numpy as np
import math
import scipy.stats as stats
from scipy.stats import ttest_ind
from scipy.special import ndtr as ndtr
from scipy import stats
import matplotlib.pyplot as plt
np.random.seed(8)

In [2]:
pip install researchpy 

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
import researchpy as rp

In [4]:
#load dataset
shark2 = pd.read_csv('Shark_Temp_Salt_Clean.csv')
shark2['month_year'] = pd.to_datetime(shark2['Date']).dt.to_period('M')
shark2.head()

,Unnamed: 0,sex,organismID,year,month,day,Date,Latitude,Longitude,Temperature,Salinity,month_year
0,0,MALE,A69-9002-15967,2012,6,7,2012-06-07,-23.44,151.90,22.686416,35.471758,2012-06
1,1,MALE,A69-9002-15967,2012,6,6,2012-06-06,-23.44,151.90,22.798087,35.464934,2012-06
2,2,MALE,A69-9002-15967,2012,3,1,2012-03-01,-23.44,151.97,26.556687,35.544015,2012-03
3,3,MALE,A69-9002-15967,2012,3,21,2012-03-21,-23.43,151.94,26.179308,35.549060,2012-03
4,4,MALE,A69-9002-15967,2012,3,27,2012-03-27,-23.43,151.94,26.164600,35.559886,2012-03


## T-test

Doing an independent sample t-test on your dataset:

In [5]:
ttemp = rp.ttest(group1= shark2['Temperature'][shark2['sex'] == 'MALE'], group1_name= "Male",
         group2= shark2['Temperature'][shark2['sex'] == 'FEMALE'], group2_name= "Female")

/home/.local/lib/python3.10/site-packages/researchpy/ttest.py:38: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  groups = group1.append(group2, ignore_index= True)


In [6]:
ttemp.summary()

AttributeError: 'tuple' object has no attribute 'summary'

In [7]:
rp.ttest(group1= shark2['Salinity'][shark2['sex'] == 'MALE'], group1_name= "Male",
         group2= shark2['Salinity'][shark2['sex'] == 'FEMALE'], group2_name= "Female")

/home/.local/lib/python3.10/site-packages/researchpy/ttest.py:38: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  groups = group1.append(group2, ignore_index= True)


(   Variable        N       Mean        SD        SE  95% Conf.   Interval
 0      Male  49907.0  35.354769  0.450714  0.002018  35.350814  35.358723
 1    Female  38135.0  35.337358  0.376425  0.001928  35.333580  35.341137
 2  combined  88042.0  35.347228  0.420238  0.001416  35.344452  35.350003,
               Independent t-test     results
 0  Difference (Male - Female) =       0.0174
 1          Degrees of freedom =   88040.0000
 2                           t =       6.0926
 3       Two side test p value =       0.0000
 4      Difference < 0 p value =       1.0000
 5      Difference > 0 p value =       0.0000
 6                   Cohen's d =       0.0414
 7                   Hedge's g =       0.0414
 8              Glass's delta1 =       0.0386
 9            Point-Biserial r =       0.0205)

Corelation

In [8]:
counts = shark2.groupby(['year','month'])['day'].count().reset_index(name='Number of sharks')
counts

,year,month,Number of sharks
0,2010,9,136
1,2010,10,57
2,2010,11,19
3,2010,12,28
4,2011,1,9
...,...,...,...
81,2021,5,1160
82,2021,6,1167
83,2021,7,1063
84,2021,8,1064


In [9]:
avg = shark2.groupby(['year','month'])['Temperature','Salinity'].mean()
avg

/tmp/ipykernel_78/559390035.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  avg = shark2.groupby(['year','month'])['Temperature','Salinity'].mean()


Temperature   Salinity
year month                        
2010 9        22.156843  35.466042
     10       22.685606  35.371902
     11       23.526308  35.361707
     12       24.986228  35.174025
2011 1        26.035397  35.011784
...                 ...        ...
2021 5        27.452791  34.749625
     6        26.447007  34.850228
     7        25.924554  34.890682
     8        25.753428  34.996751
     9        25.768170  35.044053

[86 rows x 2 columns]

In [10]:
df = pd.merge(counts,avg,on=['year','month'])
df

,year,month,Number of sharks,Temperature,Salinity
0,2010,9,136,22.156843,35.466042
1,2010,10,57,22.685606,35.371902
2,2010,11,19,23.526308,35.361707
3,2010,12,28,24.986228,35.174025
4,2011,1,9,26.035397,35.011784
...,...,...,...,...,...
81,2021,5,1160,27.452791,34.749625
82,2021,6,1167,26.447007,34.850228
83,2021,7,1063,25.924554,34.890682
84,2021,8,1064,25.753428,34.996751


In [11]:
corr = df.corr()
corr

,year,month,Number of sharks,Temperature,Salinity
year,1.000000,-0.194337,-0.254328,0.382526,-0.372846
month,-0.194337,1.000000,0.027129,-0.542842,0.470165
Number of sharks,-0.254328,0.027129,1.000000,0.013735,-0.113239
Temperature,0.382526,-0.542842,0.013735,1.000000,-0.448411
Salinity,-0.372846,0.470165,-0.113239,-0.448411,1.000000


There is no corelation between the number of sharks and the temperature and the salinity 

In [12]:
# convert object to datetime
shark2['month_year'] = shark2['month_year'].astype(str)
shark2['month_year'] = pd.to_datetime(shark2['month_year'])